In [72]:
#import statements
import psycopg2
from psycopg2.extras import execute_values
from psycopg2.extras import execute_batch
import pandas as pd, numpy as np

In [166]:
#connect to a postgresql database

#user_name: postgres
#Password: masterpw

#set up a connection to the database

def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = psycopg2.connect(
    host="database-2.cwqlksfld1r8.us-east-2.rds.amazonaws.com",
    database="ISTA_498_DB",
    user="postgres",
    password="masterpw",
    port='5432')

  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor


In [167]:
#check if the connection is successful
conn, cur = get_conn_cur()
print(conn)

<connection object at 0x7fd9b87e1430; dsn: 'user=postgres password=xxx dbname=ISTA_498_DB host=database-2.cwqlksfld1r8.us-east-2.rds.amazonaws.com port=5432', closed: 0>


In [168]:
#useful functions

def run_query(query_string):

 conn, cur = get_conn_cur() # get connection and cursor

 cur.execute(query_string) # executing string as before

 my_data = cur.fetchall() # fetch query data as before

 # here we're extracting the 0th element for each item in cur.description
 colnames = [desc[0] for desc in cur.description]

 cur.close() # close
 conn.close() # close

 return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
 conn, cur = get_conn_cur() # get connection and cursor

 # Now select column names while inserting the table name into the WERE
 column_name_query = """SELECT column_name FROM information_schema.columns
    WHERE table_name = '%s' """ %table_name

 cur.execute(column_name_query) # exectue
 my_data = cur.fetchall() # store

 cur.close() # close
 conn.close() # close

 return(my_data) # return

# Check table_names
def get_table_names():
 conn, cur = get_conn_cur() # get connection and cursor

 # query to get table names
 table_name_query = """SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public' """

 cur.execute(table_name_query) # execute
 my_data = cur.fetchall() # fetch results

 cur.close() #close cursor
 conn.close() # close connection

 return(my_data) # return your fetched results
 
# make sql_head function
def sql_head(table_name):
 conn, cur = get_conn_cur() # get connection and cursor

 # Now select column names while inserting the table name into the WERE
 head_query = """SELECT * FROM %s LIMIT 5; """ %table_name

 cur.execute(head_query) # exectue
 colnames = [desc[0] for desc in cur.description] # get column names
 my_data = cur.fetchall() # store first five rows

 cur.close() # close
 conn.close() # close

 df = pd.DataFrame(data = my_data, columns = colnames) # make into df

 return(df) # return

In [46]:
cur.execute("""CREATE TABLE aday (cloth_dec VARCHAR(255) PRIMARY KEY, price FLOAT);""")
cur.execute("""CREATE TABLE shein (cloth_dec VARCHAR(255) PRIMARY KEY, price FLOAT);""")
cur.execute("""CREATE TABLE thereFormation (cloth_dec VARCHAR(255) PRIMARY KEY, price FLOAT);""")
cur.execute("""CREATE TABLE fashionNova (cloth_dec VARCHAR(255) PRIMARY KEY, price FLOAT);""")
cur.execute("""CREATE TABLE khols (cloth_dec VARCHAR(255) PRIMARY KEY, price FLOAT);""")
cur.execute("""CREATE TABLE target (cloth_dec VARCHAR(255) PRIMARY KEY, price FLOAT);""")

DuplicateTable: relation "aday" already exists


In [61]:
#get list of table names

get_table_names()

[('aday',),
 ('shein',),
 ('thereformation',),
 ('fashionnova',),
 ('khols',),
 ('target',)]

In [158]:
#create dataframes for each table
shein_df = pd.read_csv('Shein_Cleaned.csv')
thereFormation_df = pd.read_csv('ThereFormation_Cleaned.csv')
fashionNova_df = pd.read_csv('FashionNova_Data_clean.csv')
aday_df = pd.read_csv('Aday_Cleaned.csv')
target_df = pd.read_csv('Target_Data_clean.csv')
khols_df = pd.read_csv('Khols_Data_clean.csv')

In [163]:
fashionNova_df.head()

,cloth_dec,price
0,Classic High Waist Skinny Jeans - Dark Denim,24.99
1,Classic High Waist Skinny Jeans - Medium Blue ...,24.99
2,Super High Waist Denim Skinnies - Black,24.99
3,Canopy Jeans - Black,24.99
4,Classic High Waist Skinny Jeans - Light Blue Wash,24.99


In [159]:
#change column names to be lowercase
shein_df.columns = shein_df.columns.str.lower()
thereFormation_df.columns = thereFormation_df.columns.str.lower()
fashionNova_df.columns = fashionNova_df.columns.str.lower()
aday_df.columns = aday_df.columns.str.lower()
target_df.columns = target_df.columns.str.lower()
khols_df.columns = khols_df.columns.str.lower()



In [160]:
#rename cloumn to be clothing_dec
shein_df.rename(columns={'cloth_desc': 'cloth_dec'}, inplace=True)
thereFormation_df.rename(columns={'cloth_desc': 'cloth_dec'}, inplace=True)
fashionNova_df.rename(columns={'cloth_desc': 'cloth_dec'}, inplace=True)
aday_df.rename(columns={'cloth_desc': 'cloth_dec'}, inplace=True)
target_df.rename(columns={'cloth_desc': 'cloth_dec'}, inplace=True)
khols_df.rename(columns={'cloth_desc': 'cloth_dec'}, inplace=True)


In [162]:
#drop first column of tables with duplicate index col

khols_df.drop(khols_df.columns[0], axis=1, inplace=True)
target_df.drop(target_df.columns[0], axis=1, inplace=True)
fashionNova_df.drop(fashionNova_df.columns[0], axis=1, inplace=True)

In [144]:
def insert_dataframe_into_table(conn, table_name, df):
    """
    Insert the rows of a Pandas DataFrame into a SQL table using the given connection and table name.
    If a row with a duplicate key exists, update the existing row with the new values.
    """
    cur = conn.cursor()
    columns = ",".join(df.columns)
    values = [tuple(x) for x in df.to_numpy()]
    placeholders = ",".join(["%s"] * len(df.columns))
    sql_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT (cloth_dec) DO UPDATE SET price=EXCLUDED.price"
    execute_batch(cur, sql_query, values)


In [128]:
get_column_names('shein')

[('price',), ('cloth_dec',)]

In [169]:
insert_dataframe_into_table(conn, 'shein', shein_df)
insert_dataframe_into_table(conn, 'thereformation', thereFormation_df)
insert_dataframe_into_table(conn, 'fashionnova', fashionNova_df)
insert_dataframe_into_table(conn, 'aday', aday_df)
insert_dataframe_into_table(conn, 'khols', khols_df)
insert_dataframe_into_table(conn, 'target', target_df)

In [171]:
conn.commit()

In [172]:
sql_head('shein')

,cloth_dec,price
0,Ditsy Floral Ring Linked Triangle Bikini Swimsuit,8.49
1,Shirred Bodice Allover Print Jumpsuit Without ...,14.00
2,Two Tone Contrast Binding Tie Backless Halter ...,7.00
3,Textured Knit Duster Cardigan,18.00
4,Car & Letter Graphic Drop Shoulder Tee,8.00


In [173]:
sql_head('thereformation')

,cloth_dec,price
0,Noah Short Sleeve Shirt,128.0
1,Talina Top,148.0
2,Redford Cotton Open Knit Sweater,168.0
3,Eldora Knit Top,68.0
4,Mayson Knit Dress,128.0


In [174]:
sql_head('fashionnova')

,cloth_dec,price
0,Classic High Waist Skinny Jeans - Dark Denim,24.99
1,Classic High Waist Skinny Jeans - Medium Blue ...,24.99
2,Super High Waist Denim Skinnies - Black,24.99
3,Canopy Jeans - Black,24.99
4,Classic High Waist Skinny Jeans - Light Blue Wash,24.99


In [175]:
sql_head('aday')

,cloth_dec,price
0,All Day Long Sleeve T-Shirt,75.0
1,All Weather Cardigan,200.0
2,Back To Front Shirt Dress,180.0
3,Bend and Stretch Dress Pants,160.0
4,Best In Class Bodysuit,105.0


In [176]:
sql_head('khols')

,cloth_dec,price
0,Women's Gloria Vanderbilt Amanda Classic Jeans,24.99
1,Women's Sonoma Goods For Life® Supersoft Stret...,18.79
2,Women's Sonoma Goods For Life® Supersoft Midri...,18.79
3,Women's Sonoma Goods For Life® Midrise Bootcut...,24.99
4,Women's Sonoma Goods For Life® Straight-Leg ...,18.79


In [177]:
sql_head('target')

,cloth_dec,price
0,High-Waisted Classic Leggings - Wild Fable Black,3.50
1,Tiny Tank Top - Wild Fable,2.50
2,90 Degree by Reflex - Women's Soft Comfy Loung...,10.99
3,Tonal Shamrocks St. Patrick's Day Crew Socks -...,3.50
4,Short Sleeve Casual T-Shirt - A New Day,4.40
